# AWS Lambda Operations - Comprehensive Guide

This notebook provides a complete walkthrough of AWS Lambda serverless compute operations using boto3 SDK.

## Prerequisites - Manual AWS Setup

Before running this notebook, the following setup was completed manually in AWS Console:

### Step 1: Create IAM Role for Lambda Execution
- Logged into AWS account
- Navigate to **IAM** > **Roles** > **Create role**
- Select trusted entity: **AWS service** > **Lambda**
- Attach policies:
  - **AWSLambdaBasicExecutionRole** (CloudWatch Logs access)
  - **AmazonS3FullAccess** (for S3 integration examples)
- Role name: **lambda-execution-role**
- Copy the Role ARN (needed for creating functions)

### Step 2: IAM User for Lambda Management
- Created IAM user with **programmatic access**
- Attached policies:
  - **AWSLambda_FullAccess** (manage Lambda functions)
  - **IAMReadOnlyAccess** (read roles for function creation)
- Generated access credentials (Access Key ID, Secret Access Key)
- Saved credentials securely in `.env` file

### Step 3: Create Simple Lambda Function (Manual Setup)
- Navigate to **Lambda** > **Create function**
- Function name: **hello-world-function**
- Runtime: **Python 3.12**
- Execution role: Use existing role **lambda-execution-role**
- Created function successfully
- This will be used for testing our boto3 operations

### Step 4: Environment File
Created `.env` file with the following variables:
```
AWS_ACCESS_KEY=your-access-key-id
AWS_SECRET_KEY=your-secret-access-key
AWS_REGION=us-east-2
LAMBDA_ROLE_ARN=arn:aws:iam::YOUR-ACCOUNT-ID:role/lambda-execution-role
```

### Step 5: Python Environment
- Installed boto3 library for AWS SDK
- Installed python-dotenv for secure credential loading

Now we're ready to explore Lambda operations programmatically.

---

## What You'll Learn

### Phase 1: Basic Lambda Operations (Read-Only)
- List all Lambda functions in your account
- Get function details (configuration, memory, timeout)
- Invoke functions and retrieve responses
- View function logs

### Phase 2: Function Management (Create & Deploy)
- Create Lambda functions programmatically
- Upload and update function code
- Delete functions
- Package Python code for deployment

### Phase 3: Configuration & Environment
- Set environment variables
- Configure memory and timeout settings
- Update execution role
- Manage function aliases and versions

### Phase 4: Event Sources & Triggers
- Create S3 event triggers
- Set up scheduled execution (EventBridge/CloudWatch)
- Configure API Gateway integration
- Process event data from different sources

### Phase 5: Advanced Features
- Create and attach Lambda layers
- View CloudWatch logs programmatically
- Implement error handling and retries
- Monitor function metrics
- Optimize performance and cost

## Lambda Architecture Overview

```
┌─────────────────────────────────────────────────────────┐
│  AWS LAMBDA ARCHITECTURE                                │
│                                                         │
│  Event Sources                Lambda Function           │
│  ┌──────────────┐            ┌──────────────┐          │
│  │ S3 Upload    │───────────>│              │          │
│  │ API Request  │───────────>│  Your Code   │          │
│  │ Schedule     │───────────>│  (Handler)   │          │
│  │ Manual Invoke│───────────>│              │          │
│  └──────────────┘            └──────┬───────┘          │
│                                     │                  │
│                                     ▼                  │
│  AWS Manages:              ┌──────────────┐            │
│  • Server provisioning     │   Outputs:   │            │
│  • Scaling                 │   • Return   │            │
│  • High availability       │   • S3 Write │            │
│  • Patching               │   • Database │            │
│  • Monitoring             └──────────────┘            │
│                                                         │
│  You Pay Only For:                                      │
│  • Number of requests (invocations)                     │
│  • Compute time (GB-seconds)                            │
└─────────────────────────────────────────────────────────┘
```

## Key Concepts

**Function**: Your code package with handler, runtime, and dependencies

**Handler**: Entry point function (e.g., `lambda_handler(event, context)`)

**Runtime**: Execution environment (Python 3.12, Node.js 20, Java 11, etc.)

**Event**: Input data passed to your function

**Context**: Runtime information (request ID, memory limit, etc.)

**Execution Role**: IAM role that grants permissions to your function

**Trigger**: Event source that invokes your function

**Cold Start**: First invocation after deployment (slower)

**Warm Start**: Subsequent invocations (faster, reuses container)

In [1]:
import boto3
from dotenv import load_dotenv
load_dotenv()
import os
import json

## 1. Environment Setup

Loading AWS credentials securely from environment variables using `.env` file.

**Security Best Practice:** Never hardcode credentials in your code!

In [2]:
ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
SECRET_KEY = os.getenv("AWS_SECRET_KEY")
AWS_REGION = os.getenv("AWS_REGION")
LAMBDA_ROLE_ARN = os.getenv("LAMBDA_ROLE_ARN")

## 2. Initialize Lambda Client

Create a boto3 Lambda client to interact with AWS Lambda service.

In [3]:
lambda_client = boto3.client('lambda',
                              region_name=AWS_REGION,
                              aws_access_key_id=ACCESS_KEY,
                              aws_secret_access_key=SECRET_KEY)

## PHASE 1: BASIC LAMBDA OPERATIONS

Starting with read-only operations to safely explore your Lambda functions.

### Function 1: List All Lambda Functions

Retrieve a list of all Lambda functions in your AWS account.

In [5]:
from botocore.exceptions import ClientError

def list_lambda_functions():
    """
    List all Lambda functions in your AWS account
    
    Returns:
        list: List of function names, or empty list if error
    """
    try:
        response = lambda_client.list_functions()
        
        functions = response.get('Functions', [])
        
        if not functions:
            print("No Lambda functions found in your account")
            return []
        
        print(f"Found {len(functions)} Lambda function(s):")
        
        function_names = []
        for func in functions:
            name = func['FunctionName']
            runtime = func['Runtime']
            memory = func['MemorySize']
            timeout = func['Timeout']
            last_modified = func['LastModified']
            
            print(f"Function: {name}")
            print(f"Runtime: {runtime}")
            print(f"Memory: {memory} MB")
            print(f"Timeout: {timeout} seconds")
            print(f"Last Modified: {last_modified}")
            
            function_names.append(name)
        
        return function_names
        
    except ClientError as e:
        print(f"ERROR: Failed to list functions - {e}")
        return []

# Test the function
function_names = list_lambda_functions()

No Lambda functions found in your account


### Function 2: Get Function Configuration

Retrieve detailed configuration for a specific Lambda function including code size, environment variables, and execution role.

In [6]:
def get_function_config(function_name):
    """
    Get detailed configuration for a Lambda function
    
    Args:
        function_name (str): Name of the Lambda function
    
    Returns:
        dict: Function configuration, or None if error
    """
    try:
        response = lambda_client.get_function_configuration(
            FunctionName=function_name
        )
        
        print(f"Configuration for '{function_name}':")
        
        # Basic info
        print(f"Function ARN: {response['FunctionArn']}")
        print(f"Runtime: {response['Runtime']}")
        print(f"Handler: {response['Handler']}")
        print(f"Memory Size: {response['MemorySize']} MB")
        print(f"Timeout: {response['Timeout']} seconds")
        print(f"Code Size: {response['CodeSize']:,} bytes")
        
        # Execution role
        print(f"Execution Role: {response['Role']}")
        
        # Environment variables
        env_vars = response.get('Environment', {}).get('Variables', {})
        if env_vars:
            print(f"Environment Variables:")
            for key, value in env_vars.items():
                print(f"  {key}: {value}")
        else:
            print(f"Environment Variables: None")
        
        # Layers
        layers = response.get('Layers', [])
        if layers:
            print(f"Layers:")
            for layer in layers:
                print(f"  - {layer['Arn']}")
        else:
            print(f"Layers: None")
        
        # VPC config
        vpc_config = response.get('VpcConfig', {})
        if vpc_config.get('VpcId'):
            print(f"VPC ID: {vpc_config['VpcId']}")
        else:
            print(f"VPC: Not configured")
        
        return response
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to get function config - {e}")
        return None

# Test with the first function from our list
if function_names:
    config = get_function_config(function_names[0])

### Function 3: Invoke Lambda Function

Execute a Lambda function and retrieve its response. This is the core operation for running serverless code.

In [7]:
def invoke_lambda(function_name, payload=None, invocation_type='RequestResponse'):
    """
    Invoke a Lambda function with optional payload
    
    Args:
        function_name (str): Name of the Lambda function
        payload (dict): Input data for the function (optional)
        invocation_type (str): 
            - 'RequestResponse' (default): Synchronous, wait for response
            - 'Event': Asynchronous, don't wait for response
            - 'DryRun': Validate parameters without executing
    
    Returns:
        dict: Function response including status code and payload
    """
    try:
        # Prepare payload
        if payload is None:
            payload = {}
        
        # Convert payload to JSON bytes
        payload_bytes = json.dumps(payload).encode('utf-8')
        
        print(f"Invoking '{function_name}' with invocation type: {invocation_type}")
        print(f"Payload: {json.dumps(payload, indent=2)}")
        print("-" * 80)
        
        # Invoke function
        response = lambda_client.invoke(
            FunctionName=function_name,
            InvocationType=invocation_type,
            Payload=payload_bytes
        )
        
        # Extract response details
        status_code = response['StatusCode']
        print(f"Status Code: {status_code}")
        
        # Read response payload
        if invocation_type == 'RequestResponse':
            response_payload = response['Payload'].read().decode('utf-8')
            response_data = json.loads(response_payload)
            
            print(f"Response Payload:")
            print(json.dumps(response_data, indent=2))
            
            # Check for function errors
            if 'FunctionError' in response:
                print(f"Function Error: {response['FunctionError']}")
            else:
                print(f"SUCCESS: Function executed successfully")
            
            return {
                'statusCode': status_code,
                'payload': response_data,
                'executedVersion': response.get('ExecutedVersion', '$LATEST')
            }
        else:
            print(f"Asynchronous invocation initiated")
            return {
                'statusCode': status_code,
                'message': 'Asynchronous invocation - no immediate response'
            }
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'ResourceNotFoundException':
            print(f"ERROR: Function '{function_name}' not found")
        else:
            print(f"ERROR: Failed to invoke function - {e}")
        return None

# Test with a simple payload
if function_names:
    test_payload = {
        "name": "Lambda Learner",
        "message": "Testing from boto3"
    }
    result = invoke_lambda(function_names[0], test_payload)